## In progress

In [1]:
%reset -f
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
from torch.utils.data import Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
from tsai.all import *

torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE = False
torchaudio.set_audio_backend('soundfile')
import os
import random
from pathlib import Path

import matplotlib.pyplot as plt
import psutil
import requests

from utils import *
from wingbeat_datasets import *
from wingbeat_models import *

import os
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import pytorch_lightning as pl


print(f'Total RAM      : {bytes2GB(psutil.virtual_memory().total):5.2f} GB')
print(f'Available RAM  : {bytes2GB(psutil.virtual_memory().available):5.2f} GB\n')

/home/kalfasyan/anaconda3/envs/wbai/lib/python3.6/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


Total RAM      : 31.21 GB
Available RAM  : 22.63 GB



In [21]:
class Conv1dNet(pl.LightningModule):
    def __init__(self, learning_rate=1e-2):
        super(Conv1dNet, self).__init__()
        self.learning_rate = learning_rate
        self.save_hyperparameters('learning_rate')

        self.conv1 = nn.Conv1d(1, 16, 3)
        self.bn1 = nn.BatchNorm1d(16)
        self.pool1 = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(16, 32, 3)
        self.bn2 = nn.BatchNorm1d(32)
        self.pool2 = nn.MaxPool1d(2)

        self.conv3 = nn.Conv1d(32, 64, 3)
        self.bn3 = nn.BatchNorm1d(64)
        self.pool3 = nn.MaxPool1d(2)

        self.conv4 = nn.Conv1d(64, 128, 3)
        self.bn4 = nn.BatchNorm1d(128)
        self.pool4 = nn.MaxPool1d(2)

        self.conv5 = nn.Conv1d(128, 256, 3)
        self.bn5 = nn.BatchNorm1d(256)
        self.pool5 = nn.MaxPool1d(2)    

        self.dropout = nn.Dropout()
        self.avgPool = nn.AvgPool1d(154)
        self.fc1 = nn.Linear(256, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = self.conv5(x)
        x = F.relu(self.bn5(x))
        x = self.pool5(x)
        x = self.dropout(x)
        x = self.avgPool(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        return x


    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y, p = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        # Here we just reuse the validation_step for testing
        return self.validation_step(batch, batch_idx)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer


In [22]:
num_epochs = 20
batch_size = 32
batch_size_val = batch_size * 2
validation_split = .2
shuffle_dataset = True
num_workers = psutil.cpu_count()
random_seed= 42

In [23]:
# Init our model
conv1d_model = Conv1dNet()

dmel1 = WingbeatsDataset("Melanogaster_RL/Y", custom_label=0,transform=FilterWingbeat('bandpass'))
dmel2 = WingbeatsDataset("Melanogaster_RL/Z", custom_label=0,transform=FilterWingbeat('bandpass'))

dsuz1 = WingbeatsDataset("Suzukii_RL/L", custom_label=1,transform=FilterWingbeat('bandpass'))
dsuz2 = WingbeatsDataset("Suzukii_RL/R", custom_label=1,transform=FilterWingbeat('bandpass'))

transformed_dataset = ConcatDataset([dmel1, dsuz1])

train_size = int(0.8 * len(transformed_dataset))
valid_size = len(transformed_dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(transformed_dataset, [train_size, valid_size])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size_val, num_workers=num_workers)

# Train the model ⚡
trainer = pl.Trainer(gpus=1, max_epochs=3, progress_bar_refresh_rate=20)
trainer.fit(conv1d_model, train_dataloader, valid_dataloader)

Found 29002 in dataset: Melanogaster_RL/Y, and 1 label(s): ['Y']
Label(s) changed to 0
Found 24763 in dataset: Melanogaster_RL/Z, and 1 label(s): ['Z']
Label(s) changed to 0
Found 21940 in dataset: Suzukii_RL/L, and 1 label(s): ['L']
Label(s) changed to 1


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name    | Type        | Params
-----------------------------------------
0  | conv1   | Conv1d      | 64    
1  | bn1     | BatchNorm1d | 32    
2  | pool1   | MaxPool1d   | 0     
3  | conv2   | Conv1d      | 1.6 K 
4  | bn2     | BatchNorm1d | 64    
5  | pool2   | MaxPool1d   | 0     
6  | conv3   | Conv1d      | 6.2 K 
7  | bn3     | BatchNorm1d | 128   
8  | pool3   | MaxPool1d   | 0     
9  | conv4   | Conv1d      | 24.7 K
10 | bn4     | BatchNorm1d | 256   
11 | pool4   | MaxPool1d   | 0     
12 | conv5   | Conv1d      | 98.6 K
13 | bn5     | BatchNorm1d | 512   
14 | pool5   | MaxPool1d   | 0     
15 | dropout | Dropout     | 0     
16 | avgPool | AvgPool1d   | 0     
17 | fc1     | Linear      | 514   
-----------------------------------------
132 K     Trainable params
0         Non-trainable params
132 K     Total params


Found 14348 in dataset: Suzukii_RL/R, and 1 label(s): ['R']
Label(s) changed to 1
Validation sanity check: 0it [00:00, ?it/s]

AssertionError: ==:
1
64

In [5]:
num_epochs = 20
batch_size = 32
batch_size_val = batch_size * 2
validation_split = .2
shuffle_dataset = True
num_workers = psutil.cpu_count()
random_seed= 42

In [6]:
dmel1 = WingbeatsDataset("Melanogaster_RL/Y", custom_label=0,transform=FilterWingbeat('bandpass'))
dmel2 = WingbeatsDataset("Melanogaster_RL/Z", custom_label=0,transform=FilterWingbeat('bandpass'))

dsuz1 = WingbeatsDataset("Suzukii_RL/L", custom_label=1,transform=FilterWingbeat('bandpass'))
dsuz2 = WingbeatsDataset("Suzukii_RL/R", custom_label=1,transform=FilterWingbeat('bandpass'))

Found 29002 in dataset: Melanogaster_RL/Y, and 1 label(s): ['Y']
Label(s) changed to 0
Found 24763 in dataset: Melanogaster_RL/Z, and 1 label(s): ['Z']
Label(s) changed to 0
Found 21940 in dataset: Suzukii_RL/L, and 1 label(s): ['L']
Label(s) changed to 1
Found 14348 in dataset: Suzukii_RL/R, and 1 label(s): ['R']
Label(s) changed to 1


In [7]:
transformed_dataset = ConcatDataset([dmel1, dsuz1])

train_size = int(0.8 * len(transformed_dataset))
valid_size = len(transformed_dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(transformed_dataset, [train_size, valid_size])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size_val, num_workers=num_workers)

In [25]:
model = Conv1dNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.999))
# scheduler = ReduceLROnPlateau(optimizer, monitor='valid_loss', factor=0.1, patience=3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, verbose=True)
print(model)

Conv1dNet(
  (conv1): Conv1d(1, 16, kernel_size=(3,), stride=(1,))
  (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(64, 128, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv1d(128, 256, kernel_size=(3,), stride=(1,))
 

In [26]:
# from sklearn import preprocessing
# import itertools

# le = preprocessing.LabelEncoder()
# all_labels = [transformed_dataset.datasets[i].labels for i in range(len(transformed_dataset.datasets))]
# all_labels = list(itertools.chain.from_iterable(all_labels))
# le.fit(all_labels)

In [27]:
# Choosing whether to train on a gpu
train_on_gpu = torch.cuda.is_available()
print(f'Train on gpu: {train_on_gpu}')# Number of gpus
model = model.to('cuda', dtype=torch.float)

Train on gpu: True


In [28]:
# Model training
for epoch in range(num_epochs):
    # Going through the training set
    correct_train = 0
    model.train()
    for x_batch,y_batch,path_batch in train_dataloader:        

        y_batch = torch.as_tensor(y_batch).type(torch.LongTensor)
        x_batch,y_batch = x_batch.cuda(), y_batch.cuda()
        
        optimizer.zero_grad()
        pred = model(x_batch)
        loss = criterion(pred, y_batch)
        loss.backward()
        optimizer.step()
        correct_train += (pred.argmax(axis=1) == y_batch).float().sum().item()
    train_accuracy = correct_train / (len(train_dataloader)*batch_size) * 100.
    # Going through the validation set
    correct_valid = 0
    model.eval()
    for x_batch,y_batch,path_batch in valid_dataloader:
        
        y_batch = torch.as_tensor(y_batch).type(torch.LongTensor)
        x_batch,y_batch = x_batch.cuda(), y_batch.cuda()

        pred = model(x_batch)
        val_loss = criterion(pred, y_batch)
        correct_valid += (pred.argmax(axis=1) == y_batch).float().sum().item()
    valid_accuracy = correct_valid / (len(valid_dataloader)*batch_size_val) * 100.
    scheduler.step(val_loss)
    
    # Printing results
    print(f"Epoch {epoch}: train_acc: {train_accuracy:.2f}% loss: {loss:.3f},  val_loss: {val_loss:.3f} val_acc: {valid_accuracy:.2f}%")
    
    # Saving model checkpoints
    checkpoint = {'epoch': epoch,
                  'model': model,
                  'optimizer': optimizer,
                  'lr_sched': scheduler}
    torch.save(checkpoint, f'checkpoint_{epoch}.pth')

Epoch 0: train_acc: 83.11% loss: 0.462,  val_loss: 8.475 val_acc: 57.42%
Epoch 1: train_acc: 90.52% loss: 0.414,  val_loss: 23.375 val_acc: 57.42%
Epoch 2: train_acc: 92.38% loss: 0.223,  val_loss: 20.542 val_acc: 57.42%
Epoch 3: train_acc: 93.47% loss: 0.136,  val_loss: 5.261 val_acc: 58.35%
Epoch 4: train_acc: 94.26% loss: 0.149,  val_loss: 7.223 val_acc: 57.43%
Epoch 5: train_acc: 94.78% loss: 0.155,  val_loss: 6.453 val_acc: 60.39%
Epoch 6: train_acc: 95.26% loss: 0.116,  val_loss: 1.677 val_acc: 59.24%
Epoch 7: train_acc: 95.56% loss: 0.072,  val_loss: 55.850 val_acc: 57.42%
Epoch 8: train_acc: 95.98% loss: 0.072,  val_loss: 97.844 val_acc: 57.42%
Epoch    10: reducing learning rate of group 0 to 1.0000e-03.
Epoch 9: train_acc: 96.33% loss: 0.068,  val_loss: 33.035 val_acc: 57.42%
Epoch 10: train_acc: 97.41% loss: 0.027,  val_loss: 16.312 val_acc: 62.19%
Epoch 11: train_acc: 97.86% loss: 0.036,  val_loss: 0.232 val_acc: 82.08%
Epoch 12: train_acc: 98.11% loss: 0.037,  val_loss: 0.

## Testing

In [29]:
test_dataset = ConcatDataset([dmel2, dsuz2])
test_dataloader = DataLoader(test_dataset, batch_size=batch_size_val, num_workers=num_workers)

correct_test = 0
model.eval()
for x_batch,y_batch,path_batch in test_dataloader:

    y_batch = torch.as_tensor(y_batch).type(torch.LongTensor)
    x_batch,y_batch = x_batch.cuda(), y_batch.cuda()

    pred = model(x_batch)
    val_loss = criterion(pred, y_batch)
    correct_test += (pred.argmax(axis=1) == y_batch).float().sum().item()
test_accuracy = correct_test / (len(test_dataloader)*batch_size_val) * 100.
print(test_accuracy)

89.88459967320262
